In [175]:
import gurobipy as gb
from gurobipy import GRB, quicksum
import pandas as pd

In [176]:
food_categories_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\food_categories.csv").set_index("Food_Item")
food_preferences_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\food_preferences.csv")
nutrient_content_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\nutrient_content.csv").set_index('Unnamed: 0')
nutrient_requirements_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\nutrient_requirements.csv").set_index('Nutrient')

In [177]:
# Initialize the Gurobi model
model = gb.Model("OptiDiet")

In [178]:
# Decision Variables
food_items = food_categories_df.index
x = model.addVars(food_items, name="food", vtype=GRB.CONTINUOUS)

In [179]:
# Objective Function
total_cost = quicksum(x[food] * food_categories_df.loc[food, 'Cost_per_gram'] for food in food_items)
model.setObjective(total_cost, GRB.MINIMIZE)

### Constraints

In [180]:
# 1. Nutritional Balance Constraints
for nutrient in nutrient_requirements_df.index:
    min_req = nutrient_requirements_df.loc[nutrient, 'Min_Requirement']
    max_req = nutrient_requirements_df.loc[nutrient, 'Max_Requirement']
    model.addConstr(
        quicksum(nutrient_content_df.loc[food, nutrient] * x[food] for food in food_items) >= min_req,
        f"Min_{nutrient}")
    model.addConstr(
        quicksum(nutrient_content_df.loc[food, nutrient] * x[food] for food in food_items) <= max_req,
        f"Max_{nutrient}")

In [181]:
# 2. Dietary Preferences Constraints
# Vegetarian Constraint
model.addConstr(
    quicksum(x[food] for food in food_items if food_categories_df.loc[food, 'Is_Vegetarian'] == 1) >= food_preferences_df['Veggie_grams'][0],
    "Vegetarian_Constraint"
)

# Vegan Constraint
model.addConstr(
    quicksum(x[food] for food in food_items if food_categories_df.loc[food, 'Is_Vegan'] == 1) >= food_preferences_df['Vegan_grams'][0],
    "Vegan_Constraint"
)

# Kosher Constraint
model.addConstr(
    quicksum(x[food] for food in food_items if food_categories_df.loc[food, 'Is_Kosher'] == 1) >= food_preferences_df['Kosher_grams'][0],
    "Kosher_Constraint"
)

# Halal Constraint
model.addConstr(
    quicksum(x[food] for food in food_items if food_categories_df.loc[food, 'Is_Halal'] == 1) >= food_preferences_df['Halal_grams'][0],
    "Halal_Constraint"
)

<gurobi.Constr *Awaiting Model Update*>

In [182]:
# 3. Variety Constraints
total_weekly_consumption = food_preferences_df['All_grams'][0]
model.addConstrs((x[food] <= 0.03 * total_weekly_consumption for food in food_categories_df.index), "Variety")

{'Food_1': <gurobi.Constr *Awaiting Model Update*>,
 'Food_2': <gurobi.Constr *Awaiting Model Update*>,
 'Food_3': <gurobi.Constr *Awaiting Model Update*>,
 'Food_4': <gurobi.Constr *Awaiting Model Update*>,
 'Food_5': <gurobi.Constr *Awaiting Model Update*>,
 'Food_6': <gurobi.Constr *Awaiting Model Update*>,
 'Food_7': <gurobi.Constr *Awaiting Model Update*>,
 'Food_8': <gurobi.Constr *Awaiting Model Update*>,
 'Food_9': <gurobi.Constr *Awaiting Model Update*>,
 'Food_10': <gurobi.Constr *Awaiting Model Update*>,
 'Food_11': <gurobi.Constr *Awaiting Model Update*>,
 'Food_12': <gurobi.Constr *Awaiting Model Update*>,
 'Food_13': <gurobi.Constr *Awaiting Model Update*>,
 'Food_14': <gurobi.Constr *Awaiting Model Update*>,
 'Food_15': <gurobi.Constr *Awaiting Model Update*>,
 'Food_16': <gurobi.Constr *Awaiting Model Update*>,
 'Food_17': <gurobi.Constr *Awaiting Model Update*>,
 'Food_18': <gurobi.Constr *Awaiting Model Update*>,
 'Food_19': <gurobi.Constr *Awaiting Model Update*>,
 '

In [183]:
# 4. Non-Negativity Constraints
model.addConstrs((x[food] >= 0 for food in food_categories_df.index), "Non-Negativity")

{'Food_1': <gurobi.Constr *Awaiting Model Update*>,
 'Food_2': <gurobi.Constr *Awaiting Model Update*>,
 'Food_3': <gurobi.Constr *Awaiting Model Update*>,
 'Food_4': <gurobi.Constr *Awaiting Model Update*>,
 'Food_5': <gurobi.Constr *Awaiting Model Update*>,
 'Food_6': <gurobi.Constr *Awaiting Model Update*>,
 'Food_7': <gurobi.Constr *Awaiting Model Update*>,
 'Food_8': <gurobi.Constr *Awaiting Model Update*>,
 'Food_9': <gurobi.Constr *Awaiting Model Update*>,
 'Food_10': <gurobi.Constr *Awaiting Model Update*>,
 'Food_11': <gurobi.Constr *Awaiting Model Update*>,
 'Food_12': <gurobi.Constr *Awaiting Model Update*>,
 'Food_13': <gurobi.Constr *Awaiting Model Update*>,
 'Food_14': <gurobi.Constr *Awaiting Model Update*>,
 'Food_15': <gurobi.Constr *Awaiting Model Update*>,
 'Food_16': <gurobi.Constr *Awaiting Model Update*>,
 'Food_17': <gurobi.Constr *Awaiting Model Update*>,
 'Food_18': <gurobi.Constr *Awaiting Model Update*>,
 'Food_19': <gurobi.Constr *Awaiting Model Update*>,
 '

In [184]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 364 rows, 120 columns and 14808 nonzeros
Model fingerprint: 0xfcae0afa
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 9e+04]
Presolve removed 302 rows and 0 columns
Presolve time: 0.00s
Presolved: 62 rows, 180 columns, 7344 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.101451e+04   0.000000e+00      0s
      26    5.8297027e+04   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.01 seconds (0.01 work units)
Optimal objective  5.829702700e+04


In [185]:
number_of_constraints = model.NumConstrs - len(food_items)
print(number_of_constraints)

244


In [186]:
# Output the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Food Production Cost: {model.objVal:.2f}")
    for food in food_categories_df.index:
        if x[food].x > 1e-6:  # Only print non-zero production foods
            print(f"{food}: {x[food].x} grams")
else:
    print("No optimal solution found.")

Optimal Food Production Cost: 58297.03
Food_4: 9578.019479384162 grams
Food_6: 9991.587416440878 grams
Food_15: 17310.0 grams
Food_16: 5689.005008720754 grams
Food_17: 17310.0 grams
Food_25: 17310.0 grams
Food_27: 16782.458531213088 grams
Food_30: 5399.373969438207 grams
Food_31: 17310.0 grams
Food_35: 17310.0 grams
Food_41: 8291.186969743945 grams
Food_42: 17310.0 grams
Food_44: 17310.0 grams
Food_47: 17310.0 grams
Food_48: 17310.0 grams
Food_49: 17310.0 grams
Food_50: 17310.0 grams
Food_54: 17310.0 grams
Food_55: 17310.0 grams
Food_56: 17310.0 grams
Food_58: 17310.0 grams
Food_60: 17310.0 grams
Food_61: 17310.0 grams
Food_62: 17310.0 grams
Food_64: 17310.0 grams
Food_65: 17310.0 grams
Food_68: 17310.0 grams
Food_70: 17310.0 grams
Food_76: 17310.0 grams
Food_77: 4370.79468318188 grams
Food_78: 17310.0 grams
Food_80: 17310.0 grams
Food_81: 17310.0 grams
Food_83: 17310.0 grams
Food_86: 17310.0 grams
Food_88: 17310.0 grams
Food_91: 17310.0 grams
Food_93: 17310.0 grams
Food_95: 17310.0 gr

In [187]:
halal_kosher_total = sum(x[food].x for food in food_items if food_categories_df.loc[food, 'Is_Halal'] == 1 or food_categories_df.loc[food, 'Is_Kosher'] == 1)
total_quantity = sum(x[food].x for food in food_items)
proportion_halal_kosher = halal_kosher_total / total_quantity

In [188]:
print(proportion_halal_kosher)

0.3317254311048019


In [189]:
model.getConstrs()

[<gurobi.Constr Min_Nutrient_1>,
 <gurobi.Constr Max_Nutrient_1>,
 <gurobi.Constr Min_Nutrient_2>,
 <gurobi.Constr Max_Nutrient_2>,
 <gurobi.Constr Min_Nutrient_3>,
 <gurobi.Constr Max_Nutrient_3>,
 <gurobi.Constr Min_Nutrient_4>,
 <gurobi.Constr Max_Nutrient_4>,
 <gurobi.Constr Min_Nutrient_5>,
 <gurobi.Constr Max_Nutrient_5>,
 <gurobi.Constr Min_Nutrient_6>,
 <gurobi.Constr Max_Nutrient_6>,
 <gurobi.Constr Min_Nutrient_7>,
 <gurobi.Constr Max_Nutrient_7>,
 <gurobi.Constr Min_Nutrient_8>,
 <gurobi.Constr Max_Nutrient_8>,
 <gurobi.Constr Min_Nutrient_9>,
 <gurobi.Constr Max_Nutrient_9>,
 <gurobi.Constr Min_Nutrient_10>,
 <gurobi.Constr Max_Nutrient_10>,
 <gurobi.Constr Min_Nutrient_11>,
 <gurobi.Constr Max_Nutrient_11>,
 <gurobi.Constr Min_Nutrient_12>,
 <gurobi.Constr Max_Nutrient_12>,
 <gurobi.Constr Min_Nutrient_13>,
 <gurobi.Constr Max_Nutrient_13>,
 <gurobi.Constr Min_Nutrient_14>,
 <gurobi.Constr Max_Nutrient_14>,
 <gurobi.Constr Min_Nutrient_15>,
 <gurobi.Constr Max_Nutrient_15>